In [16]:
import numpy as np

import torch
import torch.nn as nn

In [20]:
batch_size = 10
sequence_length = 50
probabilities = np.array(coherence_levels) / 100.0
coherence = np.random.choice(probabilities, batch_size)
directions = np.sign(coherence)[:, None] * (np.random.rand(batch_size, sequence_length) < np.abs(coherence)[:, None])
noise = np.random.randn(batch_size, sequence_length) * (1 - np.abs(coherence)[:, None])
signals = directions + noise
labels = (coherence > 0).astype(int)
signals.shape

(10, 50)

In [25]:
def DDM(mu, sigma, dt, sequence_length, seed=None):
    if seed is not None:
        np.random.seed(seed) 
    num_trajectories = len(mu)
    num_steps = int(sequence_length / dt)
    x = np.zeros((num_trajectories, num_steps + 1))

    for i in range(num_trajectories):
        for t in range(num_steps):
            normal_sample = np.random.normal()
            x[i, t + 1] = x[i, t] + mu[i] * dt + sigma * np.sqrt(dt) * normal_sample
    
    labels = np.sign(mu)
    return x, labels

mu = np.array([-0.64, -0.16, -0.04, 0.04, 0.16,0.64])  
sigma = 0.5
dt = 1
sequence_length = 100
x, labels = DDM(mu, sigma, dt, sequence_length, seed=42)

In [17]:

def generate_motion_data(batch_size, sequence_length, coherence_levels):
    probabilities = np.array(coherence_levels) / 100.0
    coherence = np.random.choice(probabilities, batch_size)
    directions = np.sign(coherence)[:, None] * (np.random.rand(batch_size, sequence_length) < np.abs(coherence)[:, None])
    noise = np.random.randn(batch_size, sequence_length) * (1 - np.abs(coherence)[:, None])
    signals = directions + noise
    labels = (coherence > 0).astype(int)
    return torch.tensor(signals, dtype=torch.float32), torch.tensor(labels, dtype=torch.long)

class DriftDiffusionRNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DriftDiffusionRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True, nonlinearity='relu')
        self.fc = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x, threshold=5.0):
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        out, hn = self.rnn(x, h0)  
        decision_variable = torch.zeros(x.size(0), 2)  
        hidden_activities = [] 
        for i in range(out.shape[1]):  
            time_step_output = out[:, i, :]
            hidden_activities.append(time_step_output)  
            logits = self.fc(time_step_output)
            decision_variable += logits
            if (decision_variable.max(1).values > threshold).any():
                break
        hidden_activities = torch.stack(hidden_activities, dim=1)
        return decision_variable, hidden_activities

def train_model(model, data_generator, criterion, optimizer, epochs=10, batch_size=10, sequence_length=50, save_path = 'model.pth'):
    model.train()
    for epoch in range(epochs):
        signals, labels = data_generator(batch_size, sequence_length, coherence_levels)
        optimizer.zero_grad()
        outputs, hidden_activities = model(signals.unsqueeze(-1))  
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    torch.save(model.state_dict(), save_path)

coherence_levels = [-64, -32, -16, -4, 0, 4, 16, 32, 64]
lr=1e-3
model = DriftDiffusionRNN(input_size=1, hidden_size=100, output_size=2)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

train_model(model, generate_motion_data, criterion, optimizer, epochs=1000)


Epoch [1/1000], Loss: 4.3217
Epoch [2/1000], Loss: 1.8557
Epoch [3/1000], Loss: 1.5845
Epoch [4/1000], Loss: 1.0885
Epoch [5/1000], Loss: 0.6633
Epoch [6/1000], Loss: 1.4159
Epoch [7/1000], Loss: 1.3450
Epoch [8/1000], Loss: 0.3623
Epoch [9/1000], Loss: 1.4880
Epoch [10/1000], Loss: 1.4376
Epoch [11/1000], Loss: 1.0140
Epoch [12/1000], Loss: 0.5270
Epoch [13/1000], Loss: 0.9116
Epoch [14/1000], Loss: 0.4969
Epoch [15/1000], Loss: 0.5854
Epoch [16/1000], Loss: 0.5316
Epoch [17/1000], Loss: 0.7189
Epoch [18/1000], Loss: 0.7834
Epoch [19/1000], Loss: 0.7229
Epoch [20/1000], Loss: 0.9274
Epoch [21/1000], Loss: 0.5126
Epoch [22/1000], Loss: 0.4938
Epoch [23/1000], Loss: 0.3873
Epoch [24/1000], Loss: 0.3334
Epoch [25/1000], Loss: 0.2658
Epoch [26/1000], Loss: 0.5269
Epoch [27/1000], Loss: 0.5552
Epoch [28/1000], Loss: 0.3466
Epoch [29/1000], Loss: 0.3049
Epoch [30/1000], Loss: 0.4073
Epoch [31/1000], Loss: 0.0907
Epoch [32/1000], Loss: 0.2335
Epoch [33/1000], Loss: 0.3518
Epoch [34/1000], Lo

In [18]:
def evaluate_model(model, data_generator, batch_size=10, sequence_length=50, coherence_levels=[-64, -32, -16, -4, 0, 4, 16, 32, 64]):
    model.eval()  #
    with torch.no_grad(): 
        signals, labels = data_generator(batch_size, sequence_length, coherence_levels)
        outputs, hidden_activities = model(signals.unsqueeze(-1))

        probabilities = torch.softmax(outputs, dim=1)
        predictions = torch.argmax(probabilities, dim=1)

        print(f"Predictions: {predictions}")
        print(f"Labels: {labels}")
        print(f"Hidden Activities Shape: {hidden_activities.shape}") 
        
        return predictions, hidden_activities

predictions, hidden_activities = evaluate_model(model, generate_motion_data)            #K, T, N

Predictions: tensor([0, 1, 1, 1, 1, 0, 0, 0, 0, 1])
Labels: tensor([0, 1, 1, 1, 1, 1, 0, 0, 0, 1])
Hidden Activities Shape: torch.Size([10, 50, 100])
